#Import libraries

In [1]:
!pip install folium

     |████████████████████████████████| 94 kB 4.6 MB/s  eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
import requests
import urllib
import folium    # improt the map library

from sklearn.cluster import KMeans # import k-means from clustering stage
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim      # to convert an address into longitude and latitude values
from pandas.io.json import json_normalize      # tranform JSON file into a pandas dataframe

# Matplotlib and related plot modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Scrap the data from wiki

In [3]:
#link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

raw_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
        
soup = BeautifulSoup(raw_data, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

data = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned\n":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned\n" in row[2]: 
            row[2] = row[1]
        data.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(data, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


#Remove "\n" at the end of each string in the Neighborhood column

In [4]:
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()   #Group all neighborhoods with the same postal code

df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
print("Shape: ", df.shape)

Shape:  (103, 3)
